---
format:
  html:
    code-fold: false
jupyter: python3
---

# OpenStreetMap {#sec-osm}

In [ ]:
from IPython.display import YouTubeVideo

This session is all about OpenStreetMap. The following will resources provide an overview of the OpenStreetMap project:

-   A clip

In [ ]:
#| echo: false
#| output: asis
YouTubeVideo(
    "Phwrgb16oEM", width=700, height=300
)

-   [This piece](https://joemorrison.medium.com/openstreetmap-is-having-a-moment-dcc7eef1bb01) is about how OpenStreetMap is currently being created and some of the implications this may have.
-   Anderson et al. [@anderson2019corporate] provide some of the academic underpinnings to the views expressed in the above piece.

## Packages and modules

In [ ]:
import geopandas
import contextily
import matplotlib.pyplot as plt
from IPython.display import GeoJSON

## Data

Since some of the query options we will discuss involve pre-defined extents, we will read the Madrid neighbourhoods dataset first.

Assuming you have the file locally on the path `../data/`:

In [ ]:
neis = geopandas.read_file("../data/neighbourhoods.geojson")

To make some of the examples below computationally easier on OpenStreetMap servers, we will single out the smallest neighborhood:

In [ ]:
areas = neis.to_crs(
    epsg=32630
).area

smallest = neis[areas == areas.min()]
smallest

In [ ]:
fig, ax = plt.subplots()

smallest.plot(
    facecolor="none", edgecolor="blue", linewidth=2, ax=ax
)

contextily.add_basemap(
    ax, 
    crs=smallest.crs, 
    source=contextily.providers.OpenStreetMap.Mapnik
)

plt.show()

## `osmnx`

Let's import one more package, `osmnx`, designed to easily download, model, analyse, and visualise street networks and other geospatial features from OpenStreetMap.

In [ ]:
import osmnx as ox

Here is a trick (courtesy of [Martin Fleischmann](https://martinfleischmann.net/) to pin all your queries to OpenStreetMap to a specific date, so results are always reproducible, even if the map changes in the meantime.

In [ ]:
ox.settings.overpass_settings = (
    '[out:json][timeout:90][date:"2021-03-07T00:00:00Z"]'
)

::: callout-note
Much of the methods covered here rely on the `osmnx.features` module. Check out its reference [here](https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.features).
:::

There are two broad areas to keep in mind when querying data on OpenStreetMap through `osmnx`:

-   The interface to specify the extent of the search.

-   The nature of the entities being queried. Here, the interface relies entirely on OpenStreetMap's tagging system. Given the distributed nature of the project, this is variable, but a good place to start is:

> <https://wiki.openstreetmap.org/wiki/Tags>

Generally, the interface we will follow involves the following:

```         
received_entities = ox.features_from_XXX(
    <extent>, tags={<key>: True/<value(s)>}, ...
)
```

The `<extent>` can take several forms. We can print out the available forms:

In [ ]:
[i for i in dir(ox) if "features_from_" in i]

The `tags` follow the [official feature spec](https://wiki.openstreetmap.org/wiki/Map_features).

## Buildings

In [ ]:
blgs = ox.features_from_polygon(
    smallest.squeeze().geometry, tags={"building": True}
)

In [ ]:
fig, ax = plt.subplots()

blgs.plot(ax=ax)

plt.show()

In [ ]:
blgs.info()

In [ ]:
blgs.head()

If you want to visit the entity online, you can do so at:

> `https://www.openstreetmap.org/<unique_id>`

::: callout-note
**Challenge:** Extract the building footprints for the Sol neighbourhood in `neis`.
:::

## Other polygons

In [ ]:
park = ox.features_from_place(
    "Parque El Retiro, Madrid", tags={"leisure": "park"}
)

In [ ]:
fig, ax = plt.subplots()

park.plot(
    facecolor="none", edgecolor="blue", linewidth=2, ax=ax
)

contextily.add_basemap(
    ax, 
    crs=smallest.crs, 
    source=contextily.providers.OpenStreetMap.Mapnik
)

plt.show()

## Points of interest

Bars around Atocha station:

In [ ]:
bars = ox.features_from_address(
    "Puerta de Atocha, Madrid", tags={"amenity": "bar"}, dist=1500
)

We can quickly explore with `GeoJSON`:

In [ ]:
bars.explore()

And stores within Malasaña:

In [ ]:
shops = ox.features_from_address(
    "Malasaña, Madrid, Spain", # Boundary to search within
    tags={
        "shop": True,
        "landuse": ["retail", "commercial"],
        "building": "retail"
    },
    dist=1000
)

shops.explore()

We use `features_from_place` for delineated areas ("polygonal entities"):

In [ ]:
cs = ox.features_from_place(
    "Madrid, Spain",
    tags={"amenity": "charging_station"}
)

cs.explore()

Similarly, we can work with location data. For example, searches around a given point:

In [ ]:
bakeries = ox.features_from_point(
    (40.418881103417675, -3.6920446157455444),
    tags={"shop": "bakery", "craft": "bakery"},
    dist=500
)

bakeries.explore()

::: callout-note
**Challenge:**

-   *How many music shops does OSM record within 750 metres of Puerta de Alcalá?*

\- *Are there more restaurants or clothing shops within the polygon that represents the Pacífico neighbourhood in `neis` table?*
:::

## Streets

Street data can be obtained as another type of entity, as above; or as a graph object.

### Geo-tables

In [ ]:
centro = ox.features_from_polygon(
    neis.query("neighbourhood == 'Sol'").squeeze().geometry,
    tags={"highway": True}
)

We can get a quick peak into what is returned (in grey), compared to the region we used for the query:

In [ ]:
fig, ax = plt.subplots()

neis.query(
    "neighbourhood == 'Sol'"
).plot(color="k", ax=ax)

centro.plot(
    ax=ax, 
    color="0.5", 
    linewidth=0.2, 
    markersize=0.5
)

plt.show()

This however will return all sorts of things:

In [ ]:
centro.geometry

### Spatial graphs

The `graph_from_XXX()` functions return clean, processed *graph* objects for the street network. Available options are:

In [ ]:
[i for i in dir(ox) if "graph_from_" in i]

Here is an example:

In [ ]:
centro_gr = ox.graph_from_polygon(
    neis.query("neighbourhood == 'Sol'").squeeze().geometry,
)

This is indeed a graph object (as defined by the `networkx` package):

In [ ]:
centro_gr

To visualise it, there are several plotting options:

In [ ]:
[i for i in dir(ox) if "plot_graph" in i]

For example:

In [ ]:
fig, ax = plt.subplots()

ox.plot_figure_ground(centro_gr)

In [ ]:
ox.graph_to_gdfs(centro_gr, nodes=False).explore()

::: callout-note
**Challenge:** *How many bookshops are within a 50m radious of the Paseo de la Castellana?*
:::

Bonus tip: this one involves the following steps:

-   Extracting the street segment for Paseo de la Castellana
-   Drawing a 50m buffer around it
-   Querying OSM for bookshops \`\`\`

## Next steps

If you found the content in this block useful, the following resources represent some suggestions on where to go next:

-   Parts of the block are inspired and informed by Geoff Boeing's excellent [course on Urban Data Science](https://github.com/gboeing/ppd599)
-   More in depth content about `osmnx` is available in the [official examples collection](https://github.com/gboeing/osmnx-examples)
-   Boeing (2020) {cite}`boeing2020exploring` illustrates how OpenStreetMap can be used to analyse urban form ([Open Access](https://osf.io/preprints/socarxiv/rnwgv/))